<a href="https://colab.research.google.com/github/Nitecry7/FPL-Neural-Networks-Project/blob/main/FPL_Neural_Networks_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Predicting FPL Player Points (2024–2025 Season) with Neural Networks**

**Goal**  
Given historical FPL and performance statistics for each player up to gameweek *t−1*, we want to **predict that player's FPL points in gameweek t**.

We treat each **(player, gameweek)** as one training example.

In this notebook we will:

1. **Load and understand** the FPL dataset:  
   - `players.csv`, `teams.csv`, `playerstats.csv`.

2. **Build a clean modelling table** that combines:
   - Static features: position, team, team strength, Elo rating, etc.
   - Form & performance features: lagged points, rolling averages, transfers, etc.

3. **Create time-aware splits** (train / validation / test) based on gameweeks.

4. **Train and evaluate**:
   - A simple **baseline model** (e.g. using recent average points).
   - A **Multilayer Perceptron (MLP)** regressor on tabular features.
   - An **LSTM** model using sequences of past gameweeks.
   - A **GRU** model, similar to LSTM but with a different recurrent cell.

The focus is on building a **clear, reproducible pipeline** that mirrors a typical deep-learning workflow used for time-based prediction tasks.


## Setup & Imports

In [1]:
# ==========================
# Basic setup and libraries
# ==========================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Nicer Pandas display
pd.set_option("display.max_columns", 150)
pd.set_option("display.width", 200)

# Reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

print("Libraries imported.")


Libraries imported.


## 1. Loading the Raw FPL Data

We work with three CSV files:

- **`players.csv`**  
  One row per FPL player with metadata, for example:  
  `player_id`, `player_code`, `web_name`, `team_code`, `position`, etc.

- **`teams.csv`**  
  One row per team with team-level information:  
  `id`, `code`, `name`, `short_name`, `strength_*`, `elo`, etc.

- **`playerstats.csv`**  
  One row per **(player, gameweek)** with their performance stats:  
  `player_id` (or `id`), `gw`, `event_points`, `minutes`, `ict_index`,  
  `transfers_in_event`, `transfers_out_event`, and possibly advanced stats like  
  `expected_goal_involvements_per_90`.

We will:

1. Upload these three CSVs into Colab.
2. Load them into Pandas DataFrames.
3. Inspect shapes and the first few rows to confirm structure.


In [2]:
from google.colab import files

print("Please select the CSV files (players.csv, teams.csv, playerstats.csv) to upload:")
uploaded = files.upload()

for fn in uploaded.keys():
  print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')


Please select the CSV files (players.csv, teams.csv, playerstats.csv) to upload:


Saving players.csv to players.csv
Saving playerstats.csv to playerstats.csv
Saving teams.csv to teams.csv
User uploaded file "players.csv" with length 39335 bytes
User uploaded file "playerstats.csv" with length 5723303 bytes
User uploaded file "teams.csv" with length 1363 bytes


In [3]:

players = pd.read_csv('players.csv')
teams = pd.read_csv('teams.csv')
playerstats = pd.read_csv('playerstats.csv')

print("players:", players.shape)
print("teams:", teams.shape)
print("playerstats:", playerstats.shape)

display(players.head())
display(teams.head())
display(playerstats.head())


players: (804, 7)
teams: (20, 13)
playerstats: (27657, 58)


,player_code,player_id,first_name,second_name,web_name,team_code,position
0,438098,1,Fabio,Ferreira Vieira,Fábio Vieira,3,Midfielder
1,100051017,735,Mikel,Arteta,Arteta,3,Unknown
2,616059,756,Jack,Porter,Porter,3,Goalkeeper
3,538182,790,Jimi,Gower,Gower,3,Midfielder
4,514307,793,Jack,Henry-Francis,Jack Henry-Francis,3,Midfielder


,code,id,name,short_name,strength,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id,elo
0,3,1,Arsenal,ARS,5,1350,1350,1390,1400,1310,1300,1,1991
1,7,2,Aston Villa,AVL,3,1145,1240,1130,1180,1160,1300,2,1870
2,91,3,Bournemouth,BOU,3,1170,1200,1120,1180,1220,1220,127,1806
3,94,4,Brentford,BRE,3,1130,1180,1100,1100,1160,1260,130,1809
4,36,5,Brighton,BHA,3,1140,1165,1090,1140,1190,1190,131,1825


,id,status,chance_of_playing_next_round,chance_of_playing_this_round,now_cost,now_cost_rank,now_cost_rank_type,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,selected_by_percent,selected_rank,selected_rank_type,total_points,event_points,points_per_game,points_per_game_rank,points_per_game_rank_type,bonus,bps,form,form_rank,form_rank_type,value_form,value_season,dreamteam_count,transfers_in,transfers_in_event,transfers_out,transfers_out_event,ep_next,ep_this,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,expected_goals_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,influence,influence_rank,influence_rank_type,creativity,creativity_rank,creativity_rank_type,threat,threat_rank,threat_rank_type,ict_index,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,direct_freekicks_order,penalties_order,gw,set_piece_threat
0,5,u,0.0,0.0,4.0,645,55,0,0,0,0,0.0,602,69,0,0,0.0,573,63,0,0,0.0,464,53,0.0,0.0,0,0,0,4399,20,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,566,63,0.0,549,45,0.0,512,31,0.0,566,63,NaN,NaN,NaN,23,NaN
1,6,a,100.0,100.0,5.6,100,10,0,0,1,-1,4.4,83,30,72,5,3.6,83,18,5,274,2.0,178,56,0.4,12.9,2,1220019,9591,879735,49584,2.0,2.5,0.71,1.01,1.72,14.70,0.04,0.05,0.09,0.79,270.2,168,62,220.1,122,25,172.0,130,14,66.0,136,35,NaN,NaN,NaN,23,NaN
2,7,a,100.0,100.0,4.7,323,209,0,0,-3,3,0.2,342,109,13,0,1.3,372,173,0,82,0.5,328,155,0.1,2.8,0,28956,538,46162,674,0.5,1.0,0.04,0.47,0.51,4.27,0.01,0.09,0.10,0.84,35.8,398,179,64.7,280,171,3.0,421,221,10.4,384,187,NaN,NaN,NaN,23,NaN
3,8,a,NaN,NaN,4.8,314,40,0,0,-2,2,0.2,383,132,13,0,1.6,327,106,0,39,0.0,685,241,0.0,2.7,0,25166,184,35333,549,0.0,0.5,0.00,0.03,0.03,4.45,0.00,0.01,0.01,1.24,49.2,383,133,13.7,392,135,0.0,707,249,6.3,415,139,NaN,NaN,NaN,23,NaN
4,9,a,100.0,100.0,6.7,38,19,0,0,-3,3,3.4,97,29,84,3,4.0,56,29,6,268,3.0,119,51,0.4,12.5,1,1099050,10248,1081194,41041,3.0,3.5,3.87,2.90,6.77,13.94,0.26,0.19,0.45,0.92,366.2,104,38,334.6,65,49,547.0,26,14,124.7,37,25,NaN,NaN,NaN,23,NaN


## 2. Cleaning and Aligning Player / Team IDs

We want a **single table** where each row is one **(player, gameweek)** record with:

- Player information (e.g. `web_name`, `position`, `team_code`)
- Team information (e.g. `strength`, `elo`, attack/defence strengths)
- Per-gameweek FPL stats (`event_points`, `minutes`, etc.)

ID alignment logic:

1. In `playerstats`, the column `id` often refers to the **player**.  
   → We rename `playerstats.id` → `player_id` if needed.

2. In `players`, we have:
   - `player_id` — same key as in `playerstats`
   - `team_code` — FPL team code
   - plus metadata (`web_name`, `position`, etc.)

3. In `teams`, we have:
   - `code` — FPL team code (matches `team_code`)
   - `id` — internal team ID in this dataset
   - team strength metrics and Elo rating

**Merge plan:**

1. `playerstats` **JOIN** `players` on `player_id`.  
2. The result **JOIN** `teams` on `team_code == code`.

We keep only the team columns that are useful as features.


In [4]:
# =======================================================
# 2.1 Rename playerstats.id -> player_id (if necessary)
# =======================================================
if "id" in playerstats.columns and "player_id" not in playerstats.columns:
    playerstats = playerstats.rename(columns={"id": "player_id"})

assert "player_id" in playerstats.columns, "Could not find 'player_id' in playerstats!"

# Minimal columns from players that we care about
player_cols = ["player_id", "player_code", "web_name", "team_code", "position"]
player_cols = [c for c in player_cols if c in players.columns]  # keep only existing
players_subset = players[player_cols].copy()

print("players_subset columns:", players_subset.columns.tolist())

# Merge playerstats + players on player_id
data = playerstats.merge(players_subset, on="player_id", how="left")

# Team columns we care about
team_cols = [
    "id", "code", "name", "short_name",
    "strength",
    "strength_overall_home", "strength_overall_away",
    "strength_attack_home", "strength_attack_away",
    "strength_defence_home", "strength_defence_away",
    "elo"
]
team_cols = [c for c in team_cols if c in teams.columns]
teams_subset = teams[team_cols].copy()

print("teams_subset columns:", teams_subset.columns.tolist())

# Merge with teams: player team_code vs teams.code
data = data.merge(
    teams_subset,
    left_on="team_code",
    right_on="code",
    how="left",
    suffixes=("", "_team")
)

print("Merged data shape:", data.shape)
display(data.head())


players_subset columns: ['player_id', 'player_code', 'web_name', 'team_code', 'position']
teams_subset columns: ['id', 'code', 'name', 'short_name', 'strength', 'strength_overall_home', 'strength_overall_away', 'strength_attack_home', 'strength_attack_away', 'strength_defence_home', 'strength_defence_away', 'elo']
Merged data shape: (27657, 74)


,player_id,status,chance_of_playing_next_round,chance_of_playing_this_round,now_cost,now_cost_rank,now_cost_rank_type,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,selected_by_percent,selected_rank,selected_rank_type,total_points,event_points,points_per_game,points_per_game_rank,points_per_game_rank_type,bonus,bps,form,form_rank,form_rank_type,value_form,value_season,dreamteam_count,transfers_in,transfers_in_event,transfers_out,transfers_out_event,ep_next,ep_this,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,expected_goals_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,influence,influence_rank,influence_rank_type,creativity,creativity_rank,creativity_rank_type,threat,threat_rank,threat_rank_type,ict_index,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,direct_freekicks_order,penalties_order,gw,set_piece_threat,player_code,web_name,team_code,position,id,code,name,short_name,strength,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,elo
0,5,u,0.0,0.0,4.0,645,55,0,0,0,0,0.0,602,69,0,0,0.0,573,63,0,0,0.0,464,53,0.0,0.0,0,0,0,4399,20,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,566,63,0.0,549,45,0.0,512,31,0.0,566,63,NaN,NaN,NaN,23,NaN,463748,Hein,3,Goalkeeper,1,3,Arsenal,ARS,5,1350,1350,1390,1400,1310,1300,1991
1,6,a,100.0,100.0,5.6,100,10,0,0,1,-1,4.4,83,30,72,5,3.6,83,18,5,274,2.0,178,56,0.4,12.9,2,1220019,9591,879735,49584,2.0,2.5,0.71,1.01,1.72,14.70,0.04,0.05,0.09,0.79,270.2,168,62,220.1,122,25,172.0,130,14,66.0,136,35,NaN,NaN,NaN,23,NaN,445122,J.Timber,3,Defender,1,3,Arsenal,ARS,5,1350,1350,1390,1400,1310,1300,1991
2,7,a,100.0,100.0,4.7,323,209,0,0,-3,3,0.2,342,109,13,0,1.3,372,173,0,82,0.5,328,155,0.1,2.8,0,28956,538,46162,674,0.5,1.0,0.04,0.47,0.51,4.27,0.01,0.09,0.10,0.84,35.8,398,179,64.7,280,171,3.0,421,221,10.4,384,187,NaN,NaN,NaN,23,NaN,85955,Jorginho,3,Midfielder,1,3,Arsenal,ARS,5,1350,1350,1390,1400,1310,1300,1991
3,8,a,NaN,NaN,4.8,314,40,0,0,-2,2,0.2,383,132,13,0,1.6,327,106,0,39,0.0,685,241,0.0,2.7,0,25166,184,35333,549,0.0,0.5,0.00,0.03,0.03,4.45,0.00,0.01,0.01,1.24,49.2,383,133,13.7,392,135,0.0,707,249,6.3,415,139,NaN,NaN,NaN,23,NaN,440854,Kiwior,3,Defender,1,3,Arsenal,ARS,5,1350,1350,1390,1400,1310,1300,1991
4,9,a,100.0,100.0,6.7,38,19,0,0,-3,3,3.4,97,29,84,3,4.0,56,29,6,268,3.0,119,51,0.4,12.5,1,1099050,10248,1081194,41041,3.0,3.5,3.87,2.90,6.77,13.94,0.26,0.19,0.45,0.92,366.2,104,38,334.6,65,49,547.0,26,14,124.7,37,25,NaN,NaN,NaN,23,NaN,444145,Martinelli,3,Midfielder,1,3,Arsenal,ARS,5,1350,1350,1390,1400,1310,1300,1991


## 3. Understanding the Merged Table & Sanity Checks

After merging, `data` should have:

- One row per **(player, gameweek)**.
- Key columns such as:
  - `player_id`, `web_name` — to identify the player.
  - `gw` — gameweek number.
  - `event_points` — FPL points scored in that gameweek.
  - `minutes` — how long the player played.
  - Optional advanced stats: `ict_index`, `expected_goal_involvements_per_90`, etc.
  - Team features: `strength_*`, `elo`, etc.

We will now:

1. Confirm that **`gw`** and **`event_points`** exist.
2. Ensure `gw` and `event_points` are numeric.
3. Remove rows with missing `gw` or `event_points` (usually future or invalid rows).
4. Look at the time series for one example player to confirm the structure.
5. Quickly inspect missing values in a few important numeric columns.


In [5]:
# Show columns
print("Columns in merged data:")
print(sorted(data.columns))

# Make sure key columns exist
assert "gw" in data.columns, "Expected column 'gw' (gameweek) not found!"
assert "event_points" in data.columns, "Expected column 'event_points' not found!"

# Ensure gw and event_points are numeric
data["gw"] = pd.to_numeric(data["gw"], errors="coerce")
data["event_points"] = pd.to_numeric(data["event_points"], errors="coerce")

# Drop rows without gw or event_points
data = data.dropna(subset=["gw", "event_points"]).copy()
data["gw"] = data["gw"].astype(int)

print("After cleaning gw/event_points, data shape:", data.shape)

# Inspect one example player's time series
inspect_cols = ["player_id", "web_name", "gw", "event_points", "minutes", "form", "ict_index"]
inspect_cols = [c for c in inspect_cols if c in data.columns]

example_player_id = data["player_id"].iloc[0]
print("Example player_id:", example_player_id)

display(
    data.loc[data["player_id"] == example_player_id, inspect_cols]
        .sort_values("gw")
        .head(10)
)

# Quick missing-values check in a few numeric columns
numeric_cols_to_check = ["event_points", "minutes", "ict_index"]
numeric_cols_to_check = [c for c in numeric_cols_to_check if c in data.columns]

print("\nMissing values in key numeric columns:")
print(data[numeric_cols_to_check].isna().sum())


Columns in merged data:
['bonus', 'bps', 'chance_of_playing_next_round', 'chance_of_playing_this_round', 'code', 'corners_and_indirect_freekicks_order', 'cost_change_event', 'cost_change_event_fall', 'cost_change_start', 'cost_change_start_fall', 'creativity', 'creativity_rank', 'creativity_rank_type', 'direct_freekicks_order', 'dreamteam_count', 'elo', 'ep_next', 'ep_this', 'event_points', 'expected_assists', 'expected_assists_per_90', 'expected_goal_involvements', 'expected_goal_involvements_per_90', 'expected_goals', 'expected_goals_conceded', 'expected_goals_conceded_per_90', 'expected_goals_per_90', 'form', 'form_rank', 'form_rank_type', 'gw', 'ict_index', 'ict_index_rank', 'ict_index_rank_type', 'id', 'influence', 'influence_rank', 'influence_rank_type', 'name', 'now_cost', 'now_cost_rank', 'now_cost_rank_type', 'penalties_order', 'player_code', 'player_id', 'points_per_game', 'points_per_game_rank', 'points_per_game_rank_type', 'position', 'selected_by_percent', 'selected_rank',

,player_id,web_name,gw,event_points,form,ict_index
18646,5,Hein,1,0,0.0,0.0
18019,5,Hein,2,0,0.0,0.0
17366,5,Hein,3,0,0.0,0.0
16709,5,Hein,4,0,0.0,0.0
16050,5,Hein,5,0,0.0,0.0
15398,5,Hein,6,0,0.0,0.0
14727,5,Hein,7,0,0.0,0.0
12172,5,Hein,8,0,0.0,0.0
14733,5,Hein,9,0,0.0,0.0
12210,5,Hein,10,0,0.0,0.0



Missing values in key numeric columns:
event_points    0
ict_index       0
dtype: int64


## 4. Step 2 — Feature Engineering: Lag and Rolling Form Features

Neural networks work best when we give them **meaningful, informative inputs**.

Instead of only using raw stats from the current gameweek, we want to encode a notion of **recent form**:

- Is the player currently performing well?
- Are they consistently getting minutes?
- Are FPL managers buying or selling them heavily?

To capture this, we:

1. **Sort** the table by `player_id` and `gw` so that each player's matches are in time order.
2. For each player separately, create **lagged and rolling features** such as:
   - `event_points_prev1`: FPL points in the **previous gameweek** (t−1).
   - `avg_points_last3`: average FPL points over the **last 3 gameweeks**, using only past data.
   - `avg_xgi_per90_last3`: rolling mean of `expected_goal_involvements_per_90` over the last 3 GWs (if present).
   - `avg_ict_last3`: rolling mean of `ict_index` over the last 3 GWs (if present).
   - `net_transfers_event`: `transfers_in_event - transfers_out_event` (how popular the player is for this GW).
   - `minutes_prev1`: minutes played in the previous GW.

3. **Filter out early gameweeks** where we do not have enough history (e.g. fewer than 3 previous GWs).

The resulting DataFrame `modelling_data` will be the **core table** used later for:

- The **baseline** model,
- The **MLP** (using tabular features),
- And building sequences for **LSTM** and **GRU**.


In [6]:
# ===============================================
# 4.1 Create lagged & rolling form features
# ===============================================

# Sort by player and gameweek to respect time order
data = data.sort_values(["player_id", "gw"]).copy()

# Group by player for time-based operations
grouped = data.groupby("player_id", group_keys=False)

# 1) Previous gameweek points
data["event_points_prev1"] = grouped["event_points"].shift(1)

# 2) Rolling average of last 3 gameweeks' points (using only past data)
data["avg_points_last3"] = (
    grouped["event_points"]
    .shift(1)                        # shift by 1 to avoid including current gw
    .rolling(window=3, min_periods=3)
    .mean()
)

# 3) Rolling average of xGI per 90 over last 3 GWs (optional)
if "expected_goal_involvements_per_90" in data.columns:
    data["avg_xgi_per90_last3"] = (
        grouped["expected_goal_involvements_per_90"]
        .shift(1)
        .rolling(window=3, min_periods=3)
        .mean()
    )
else:
    print("Column 'expected_goal_involvements_per_90' not found; skipping avg_xgi_per90_last3.")

# 4) Rolling average of ICT index over last 3 GWs (optional)
if "ict_index" in data.columns:
    data["avg_ict_last3"] = (
        grouped["ict_index"]
        .shift(1)
        .rolling(window=3, min_periods=3)
        .mean()
    )
else:
    print("Column 'ict_index' not found; skipping avg_ict_last3.")

# 5) Net transfers this event (no lag needed; it is pre-GW information)
if "transfers_in_event" in data.columns and "transfers_out_event" in data.columns:
    data["net_transfers_event"] = (
        data["transfers_in_event"] - data["transfers_out_event"]
    )
else:
    print("Transfers columns not fully available; skipping net_transfers_event.")

# 6) Previous minutes played
if "minutes" in data.columns:
    data["minutes_prev1"] = grouped["minutes"].shift(1)

# ===============================================
# 4.2 Filter to rows with enough history
# ===============================================

# We require at least:
# - previous gameweek points
# - last 3 games' average points
required_cols = ["event_points_prev1", "avg_points_last3"]

# If we created extra rolling features, we can also require them to be non-null
extra_required_cols = []
if "avg_xgi_per90_last3" in data.columns:
    extra_required_cols.append("avg_xgi_per90_last3")
if "avg_ict_last3" in data.columns:
    extra_required_cols.append("avg_ict_last3")

all_required = required_cols + extra_required_cols

modelling_data = data.dropna(subset=all_required).copy()

print("Full data shape      :", data.shape)
print("Modelling data shape :", modelling_data.shape)
print("Required feature columns:", all_required)

# Peek at a few rows to verify the new features
cols_to_show = [
    "player_id", "web_name", "gw",
    "event_points", "event_points_prev1", "avg_points_last3"
]
for c in ["avg_xgi_per90_last3", "avg_ict_last3", "net_transfers_event", "minutes_prev1"]:
    if c in modelling_data.columns:
        cols_to_show.append(c)

display(
    modelling_data[cols_to_show]
        .head(10)
)


Full data shape      : (27657, 79)
Modelling data shape : (25251, 79)
Required feature columns: ['event_points_prev1', 'avg_points_last3', 'avg_xgi_per90_last3', 'avg_ict_last3']


,player_id,web_name,gw,event_points,event_points_prev1,avg_points_last3,avg_xgi_per90_last3,avg_ict_last3,net_transfers_event
16705,1,Fábio Vieira,4,0,0.0,0.0,0.0,0.0,-71
16046,1,Fábio Vieira,5,0,0.0,0.0,0.0,0.0,-35
11794,1,Fábio Vieira,6,0,0.0,0.0,0.0,0.0,-40
14723,1,Fábio Vieira,7,0,0.0,0.0,0.0,0.0,-13
11695,1,Fábio Vieira,8,0,0.0,0.0,0.0,0.0,-14
14729,1,Fábio Vieira,9,0,0.0,0.0,0.0,0.0,-5
12192,1,Fábio Vieira,10,0,0.0,0.0,0.0,0.0,-17
19920,1,Fábio Vieira,11,0,0.0,0.0,0.0,0.0,-6
21275,1,Fábio Vieira,12,0,0.0,0.0,0.0,0.0,-5
21270,1,Fábio Vieira,13,0,0.0,0.0,0.0,0.0,-6


## 5. Building the Modelling Matrix & Time-Aware Splits

We now have `modelling_data`, where each row is one **(player, gameweek)** with:

- Target: `event_points` (FPL points in that gameweek).
- Engineered form features:  
  - `event_points_prev1`, `avg_points_last3`,  
  - `avg_xgi_per90_last3`, `avg_ict_last3`,  
  - `net_transfers_event`, `minutes_prev1`.
- Team strength features (`strength`, `elo`, etc.).
- Player metadata (`position`, `short_name`).

To feed this into neural networks, we need to:

1. **Choose input features**:
   - Numeric performance features (form, minutes, cost, transfers, etc.).
   - Team strength features (`strength`, `elo`, ...).
   - One-hot encodings for:
     - Player **position** (e.g. GK, DEF, MID, FWD).
     - Team (**short_name**, e.g. ARS, CHE, etc.).

2. **Define the target**:
   - `y = event_points` for that gameweek.

3. **Create time-aware splits**:
   - We sort unique gameweeks and split them into:
     - **Train**: earliest ~60% of gameweeks.
     - **Validation**: next ~20%.
     - **Test**: final ~20%.
   - This ensures we **only predict future gameweeks from past ones** (no data leakage).

4. **Scale the input features**:
   - Neural networks usually train better if numeric features are on similar scales.
   - We will:
     - Fit a `StandardScaler` on the **training** features only.
     - Apply the same scaling to validation and test sets.

The result of this step will be:

- `X_train_scaled`, `y_train`
- `X_val_scaled`, `y_val`
- `X_test_scaled`, `y_test`

These will be used by the **baseline**, **MLP**, **LSTM**, and **GRU** models later.


In [7]:
# ============================================
# 5.1: Choose features and encode categoricals
# ============================================
from sklearn.preprocessing import StandardScaler
import numpy as np

# Make a copy to avoid confusion
df = modelling_data.copy()

# ---- Numeric base features (only those that actually exist) ----
base_feature_candidates = [
    # Engineered form features
    "event_points_prev1", "avg_points_last3",
    "avg_xgi_per90_last3", "avg_ict_last3",
    "net_transfers_event", "minutes_prev1",

    # Raw FPL numeric features that are reasonable as inputs
    "now_cost", "selected_by_percent",
    "transfers_in_event", "transfers_out_event",
    "value_form", "value_season",
    "form", "ict_index",

    # Team strength / Elo
    "strength",
    "strength_overall_home", "strength_overall_away",
    "strength_attack_home", "strength_attack_away",
    "strength_defence_home", "strength_defence_away",
    "elo",
]

numeric_feature_cols = [c for c in base_feature_candidates if c in df.columns]
print("Numeric feature columns used:", numeric_feature_cols)

# ---- One-hot encode CATEGORICAL features: position, team ----
cat_feature_cols = []
# Position (GK / DEF / MID / FWD)
if "position" in df.columns:
    pos_dummies = pd.get_dummies(df["position"].astype(str), prefix="pos")
    df = pd.concat([df, pos_dummies], axis=1)
    cat_feature_cols.extend(pos_dummies.columns.tolist())
else:
    print("Column 'position' not found; skipping position one-hot encoding.")

# Team (short_name, e.g. ARS, CHE, etc.)
if "short_name" in df.columns:
    team_dummies = pd.get_dummies(df["short_name"].astype(str), prefix="team")
    df = pd.concat([df, team_dummies], axis=1)
    cat_feature_cols.extend(team_dummies.columns.tolist())
else:
    print("Column 'short_name' not found; skipping team one-hot encoding.")

print("Categorical (one-hot) feature columns:", cat_feature_cols)

# Full feature list
feature_cols = numeric_feature_cols + cat_feature_cols
print("Total number of features:", len(feature_cols))

# Fill any remaining NaNs in feature columns with 0
df[feature_cols] = df[feature_cols].fillna(0.0)

# ============================================
# 5.2: Define target and time-aware splits
# ============================================
target_col = "event_points"

# Sort unique gameweeks
unique_gws = np.sort(df["gw"].unique())
n_gw = len(unique_gws)
print("Unique gameweeks:", unique_gws)
print("Number of gameweeks:", n_gw)

# Time-based split: 60% train, 20% val, 20% test
train_end_idx = int(0.6 * n_gw)     # exclusive index for train
val_end_idx = int(0.8 * n_gw)       # exclusive index for val

train_gws = unique_gws[:train_end_idx]
val_gws   = unique_gws[train_end_idx:val_end_idx]
test_gws  = unique_gws[val_end_idx:]

print(f"Train GWs: {train_gws}")
print(f"Val   GWs: {val_gws}")
print(f"Test  GWs: {test_gws}")

# Build dataframes for each split
train_df = df[df["gw"].isin(train_gws)].copy()
val_df   = df[df["gw"].isin(val_gws)].copy()
test_df  = df[df["gw"].isin(test_gws)].copy()

print("train_df shape:", train_df.shape)
print("val_df shape  :", val_df.shape)
print("test_df shape :", test_df.shape)

# Features (X) and target (y)
X_train = train_df[feature_cols].values
y_train = train_df[target_col].values

X_val = val_df[feature_cols].values
y_val = val_df[target_col].values

X_test = test_df[feature_cols].values
y_test = test_df[target_col].values

print("Raw shapes:")
print("X_train:", X_train.shape, "| y_train:", y_train.shape)
print("X_val  :", X_val.shape, "| y_val  :", y_val.shape)
print("X_test :", X_test.shape, "| y_test :", y_test.shape)

# ============================================
# 5.3: Standardise features (fit on TRAIN only)
# ============================================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled   = scaler.transform(X_val)
X_test_scaled  = scaler.transform(X_test)

print("Scaled shapes:")
print("X_train_scaled:", X_train_scaled.shape)
print("X_val_scaled  :", X_val_scaled.shape)
print("X_test_scaled :", X_test_scaled.shape)


Numeric feature columns used: ['event_points_prev1', 'avg_points_last3', 'avg_xgi_per90_last3', 'avg_ict_last3', 'net_transfers_event', 'now_cost', 'selected_by_percent', 'transfers_in_event', 'transfers_out_event', 'value_form', 'value_season', 'form', 'ict_index', 'strength', 'strength_overall_home', 'strength_overall_away', 'strength_attack_home', 'strength_attack_away', 'strength_defence_home', 'strength_defence_away', 'elo']
Categorical (one-hot) feature columns: ['pos_Defender', 'pos_Forward', 'pos_Goalkeeper', 'pos_Midfielder', 'pos_Unknown', 'team_ARS', 'team_AVL', 'team_BHA', 'team_BOU', 'team_BRE', 'team_CHE', 'team_CRY', 'team_EVE', 'team_FUL', 'team_IPS', 'team_LEI', 'team_LIV', 'team_MCI', 'team_MUN', 'team_NEW', 'team_NFO', 'team_SOU', 'team_TOT', 'team_WHU', 'team_WOL']
Total number of features: 46
Unique gameweeks: [ 4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27
 28 29 30 31 32 33 34 35 36 37 38]
Number of gameweeks: 35
Train GWs: [ 4  5  6  7  

## 6. Baseline Model — Simple Heuristic from Recent Form

Before we train any neural networks, we need a **simple baseline** to compare against.

The goal is to answer:
> “Is our MLP/LSTM/GRU actually learning something useful, or just doing what a simple heuristic could do?”

We’ll build a baseline that uses **recent form only**, without any learning:

- For each (player, gameweek), we predict:
  - **Baseline prediction = `avg_points_last3`**
    - This is the average FPL points of the player over the **previous 3 gameweeks**, shifted so it only uses *past* data.
- If, for any reason, `avg_points_last3` is missing:
  - We fall back to `event_points_prev1` (points in the previous GW).
  - If that is also missing, we fall back to the **global mean** of `event_points` in the training set.

We will evaluate this baseline on:

- **Train set**
- **Validation set**
- **Test set**

using:

- **MAE (Mean Absolute Error)** — average absolute difference between predicted and true points.
- **RMSE (Root Mean Squared Error)** — penalises larger errors more.

These scores will serve as the **reference level**.  
Our neural models (MLP, LSTM, GRU) should ideally achieve **lower MAE/RMSE** than this baseline.


In [11]:
# ============================================
# 6. Baseline model: predict avg_points_last3
# (fixed evaluate_regression for older sklearn)
# ============================================

from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Safety check: ensure the columns we need exist
required_cols_for_baseline = ["avg_points_last3", "event_points_prev1", "event_points"]
for c in required_cols_for_baseline:
    assert c in train_df.columns, f"Column '{c}' missing in train_df!"

# Global mean of event_points (train only) as a final fallback
global_mean_points = train_df["event_points"].mean()
print("Global mean of event_points (train set):", global_mean_points)

def build_baseline_predictions(df_in):
    """
    Returns baseline predictions for a given dataframe:
    - primary: avg_points_last3
    - fallback 1: event_points_prev1
    - fallback 2: global mean (from train set)
    """
    preds = df_in["avg_points_last3"].copy()

    # Fallback 1: where avg_points_last3 is NaN, use event_points_prev1
    mask_nan = preds.isna()
    if mask_nan.any():
        preds[mask_nan] = df_in.loc[mask_nan, "event_points_prev1"]

    # Fallback 2: if still NaN, use global mean
    mask_nan2 = preds.isna()
    if mask_nan2.any():
        preds[mask_nan2] = global_mean_points

    return preds.values

# Build predictions for each split
baseline_train_pred = build_baseline_predictions(train_df)
baseline_val_pred   = build_baseline_predictions(val_df)
baseline_test_pred  = build_baseline_predictions(test_df)

y_train_true = y_train
y_val_true   = y_val
y_test_true  = y_test

# Helper to compute MAE and RMSE (no 'squared' argument)
def evaluate_regression(y_true, y_pred, label=""):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)  # plain MSE
    rmse = np.sqrt(mse)                       # RMSE by hand
    print(f"{label} MAE : {mae:.4f}")
    print(f"{label} RMSE: {rmse:.4f}")
    return mae, rmse

print("=== Baseline performance (predict avg_points_last3) ===")
baseline_train_mae, baseline_train_rmse = evaluate_regression(
    y_train_true, baseline_train_pred, label="Train"
)
baseline_val_mae, baseline_val_rmse = evaluate_regression(
    y_val_true, baseline_val_pred, label="Val"
)
baseline_test_mae, baseline_test_rmse = evaluate_regression(
    y_test_true, baseline_test_pred, label="Test"
)


Global mean of event_points (train set): 1.2046283309957924
=== Baseline performance (predict avg_points_last3) ===
Train MAE : 1.1262
Train RMSE: 2.2445
Val MAE : 1.1286
Val RMSE: 2.3172
Test MAE : 1.1336
Test RMSE: 2.3571


## 7. Multilayer Perceptron (MLP) – Neural Network on Tabular Features

Now that we have:

- A clean modelling table `modelling_data`
- Engineered **form features** (e.g. `event_points_prev1`, `avg_points_last3`, `avg_xgi_per90_last3`, `avg_ict_last3`)
- Encoded **categorical features** (position, team) and
- Scaled numerical inputs: `X_train_scaled`, `X_val_scaled`, `X_test_scaled`
- Targets: `y_train`, `y_val`, `y_test` (the FPL points `event_points`)

we can move from the simple baseline to our **first neural network model**.

### 7.1 Why an MLP?

A **Multilayer Perceptron (MLP)** is a fully connected neural network that is well-suited for **tabular data**:

- It can learn **non-linear interactions** between features (e.g. form × team strength × position).
- It works directly on the feature vector we constructed for each (player, gameweek).
- It is a good step up from the baseline while still being relatively simple.

In this section, we:

1. **Build** an MLP that takes the 46-dimensional feature vector as input.
2. Use:
   - 2 hidden layers with ReLU activations (e.g. 128 and 64 units).
   - **Batch Normalization** to stabilise training.
   - **Dropout** to reduce overfitting.
3. **Train** the model on the training set and validate on the validation set.
4. **Evaluate** its performance (MAE and RMSE) on train/val/test.
5. **Plot** the training vs validation MAE over epochs.

### 7.2 MLP architecture and training setup

- **Input dimension**: number of features in `X_train_scaled` (e.g. 46).
- **Hidden layers**: [128, 64] units, ReLU activation.
- **Output layer**: 1 neuron, linear activation → predicts FPL points.
- **Loss**: Mean Squared Error (MSE).
- **Metric**: Mean Absolute Error (MAE) – easier to interpret in “points”.
- **Optimiser**: Adam, learning rate = 1e-3.
- **Callbacks**:
  - `EarlyStopping(monitor='val_mae', patience=5, restore_best_weights=True)` to stop when validation MAE stops improving.
  - `ReduceLROnPlateau` to reduce learning rate when validation MAE plateaus.

We still use the same `evaluate_regression` helper function to compute MAE and RMSE,
so we can **compare directly** with the baseline.


X_train_scaled shape: (14260, 46)
X_val_scaled shape  : (5421, 46)
X_test_scaled shape : (5570, 46)
y_train shape       : (14260,)
y_val shape         : (5421,)
y_test shape        : (5570,)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         6,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,105 (59.00 KB)

 Trainable params: 14,721 (57.50 KB)

 Non-trainable params: 384 (1.50 KB)

Epoch 1/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 10s 30ms/step - loss: 5.3763 - mae: 1.5571 - val_loss: 5.2596 - val_mae: 0.9548 - learning_rate: 0.0010
Epoch 2/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 2.4363 - mae: 1.0530 - val_loss: 16.0458 - val_mae: 1.1391 - learning_rate: 0.0010
Epoch 3/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 1.9150 - mae: 0.9157 - val_loss: 23.7822 - val_mae: 1.2389 - learning_rate: 0.0010
Epoch 4/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 1.7057 - mae: 0.8424 - val_loss: 27.6100 - val_mae: 1.2880 - learning_rate: 0.0010
Epoch 5/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 1.5559 - mae: 0.7953 - val_loss: 34.6934 - val_mae: 1.3690 - learning_rate: 5.0000e-04
Epoch 6/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 1.4869 - mae: 0.7672 - val_loss: 33.5111 - val_mae: 1.3490 - learning_rate: 5.0000e-04

=== MLP performance ===
Train set:
446/446 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Train MAE : 0.8301
Train RMSE: 1.6701

Validation se

In [12]:
# ============================================
# 7. MLP model: Dense NN on tabular features
# ============================================

from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# Sanity check: shapes of our data
print("X_train_scaled shape:", X_train_scaled.shape)
print("X_val_scaled shape  :", X_val_scaled.shape)
print("X_test_scaled shape :", X_test_scaled.shape)
print("y_train shape       :", y_train.shape)
print("y_val shape         :", y_val.shape)
print("y_test shape        :", y_test.shape)

# ----------------------------
# 7.1 Build the MLP model
# ----------------------------

def build_mlp_model(input_dim, hidden_units=[128, 64], dropout_rate=0.2):
    """
    Simple feed-forward MLP for regression on tabular FPL features.

    input_dim:      number of input features (e.g. 46)
    hidden_units:   list with the number of units in each hidden layer
    dropout_rate:   dropout probability for regularisation
    """
    model = keras.Sequential()

    # Input layer
    model.add(layers.Input(shape=(input_dim,)))

    # Hidden layers
    for units in hidden_units:
        model.add(layers.Dense(units, activation="relu"))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(dropout_rate))

    # Output layer: 1 value = predicted FPL points
    model.add(layers.Dense(1, activation="linear"))

    # Compile with MSE loss and MAE metric
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss="mse",
        metrics=[keras.metrics.MeanAbsoluteError(name="mae")]
    )

    return model

input_dim = X_train_scaled.shape[1]

# For reproducibility
np.random.seed(42)
tf.random.set_seed(42)

mlp_model = build_mlp_model(input_dim)
mlp_model.summary()

# ----------------------------
# 7.2 Training setup
# ----------------------------

early_stop = keras.callbacks.EarlyStopping(
    monitor="val_mae",
    patience=5,
    restore_best_weights=True
)

reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor="val_mae",
    factor=0.5,
    patience=3,
    min_lr=1e-5
)

# ----------------------------
# 7.3 Train the MLP
# ----------------------------

history = mlp_model.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=50,
    batch_size=128,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

# ----------------------------
# 7.4 Evaluate the MLP
# ----------------------------

print("\n=== MLP performance ===")

print("Train set:")
mlp_train_pred = mlp_model.predict(X_train_scaled).ravel()
mlp_train_mae, mlp_train_rmse = evaluate_regression(
    y_train, mlp_train_pred, label="Train"
)

print("\nValidation set:")
mlp_val_pred = mlp_model.predict(X_val_scaled).ravel()
mlp_val_mae, mlp_val_rmse = evaluate_regression(
    y_val, mlp_val_pred, label="Val"
)

print("\nTest set:")
mlp_test_pred = mlp_model.predict(X_test_scaled).ravel()
mlp_test_mae, mlp_test_rmse = evaluate_regression(
    y_test, mlp_test_pred, label="Test"
)

# ----------------------------
# 7.5 Plot learning curves
# ----------------------------

plt.figure(figsize=(8, 4))
plt.plot(history.history["mae"], label="Train MAE")
plt.plot(history.history["val_mae"], label="Val MAE")
plt.xlabel("Epoch")
plt.ylabel("MAE")
plt.title("MLP training vs validation MAE")
plt.legend()
plt.grid(True)
plt.show()


## 8. Preparing Sequence Data for LSTM / GRU

The MLP treated each (player, gameweek) as an **independent sample** with a flat
feature vector.

However, football form is inherently **sequential**:

- A player’s performance in gameweek *t* depends on what happened in the last
  few gameweeks (*t−1, t−2, …*).
- Recurrent models like **LSTM** and **GRU** are designed to work with
  **time-series / sequence data**.

To use these models, we need to convert our table into sequences:

### 8.1 Sequence definition

We choose a **history window** of length `T = 5` gameweeks:

For each player and each target gameweek `t` where we have at least 5 past
observations, we build:

- **Input sequence**:  
  Features from gameweeks `[t−5, t−4, t−3, t−2, t−1]`
- **Target**:  
  `event_points` in gameweek `t`

So each training example becomes:

> shape: `(T, D)` → 5 timesteps × 46 features,  
> target: scalar FPL points in the next gameweek.

### 8.2 Train / Val / Test split for sequences

We **keep the same time-based split** as before:

- If the **target gameweek `t`** is in:
  - `train_gws` → the sequence goes into the **training set**.
  - `val_gws` → the sequence goes into the **validation set**.
  - `test_gws` → the sequence goes into the **test set**.

Important:

- For validation and test sequences, we are allowed to use **history from
  earlier train gameweeks** inside the input window, since that is past
  information.
- We never use **future data** to predict the past, so there is no leakage.

We will create:

- `X_train_seq`, `y_train_seq`
- `X_val_seq`, `y_val_seq`
- `X_test_seq`, `y_test_seq`

where each `X_*_seq` has shape:

> `(num_sequences, T, num_features)`


In [14]:
# ============================================================
# 8. Build sequence datasets for LSTM / GRU (history length T)
# ============================================================

SEQ_LEN = 5  # number of previous gameweeks to use as history

# IMPORTANT: use the same 'df' that already has:
# - numeric features
# - one-hot encoded position/team columns
seq_df = df.copy().sort_values(["player_id", "gw"]).reset_index(drop=True)

print("Full seq_df shape:", seq_df.shape)

# Use the same scaler (fitted on train) to transform ALL rows
X_all_scaled = scaler.transform(seq_df[feature_cols])  # no .values needed
y_all = seq_df["event_points"].values.astype(np.float32)
gw_all = seq_df["gw"].values
player_all = seq_df["player_id"].values

num_features = X_all_scaled.shape[1]
print("Number of features (D):", num_features)

# Convert GW lists to sets for fast membership checks
train_gws_set = set(train_gws.tolist() if hasattr(train_gws, "tolist") else list(train_gws))
val_gws_set   = set(val_gws.tolist()   if hasattr(val_gws, "tolist")   else list(val_gws))
test_gws_set  = set(test_gws.tolist()  if hasattr(test_gws, "tolist")  else list(test_gws))

X_train_seq, y_train_seq = [], []
X_val_seq,   y_val_seq   = [], []
X_test_seq,  y_test_seq  = [], []

# ------------------------------------------------------------
# For each player, slide a window of length SEQ_LEN
# ------------------------------------------------------------
for pid, group in seq_df.groupby("player_id"):
    group = group.sort_values("gw")
    idx = group.index.to_list()

    # Need at least SEQ_LEN+1 rows to form one (history, target) pair
    if len(idx) <= SEQ_LEN:
        continue

    for j in range(SEQ_LEN, len(idx)):
        target_idx = idx[j]
        target_gw = gw_all[target_idx]

        # Input sequence uses the previous SEQ_LEN gameweeks
        seq_indices = idx[j - SEQ_LEN:j]
        X_seq = X_all_scaled[seq_indices, :]         # shape: (SEQ_LEN, D)
        y_target = y_all[target_idx]                 # scalar

        if target_gw in train_gws_set:
            X_train_seq.append(X_seq)
            y_train_seq.append(y_target)
        elif target_gw in val_gws_set:
            X_val_seq.append(X_seq)
            y_val_seq.append(y_target)
        elif target_gw in test_gws_set:
            X_test_seq.append(X_seq)
            y_test_seq.append(y_target)
        # else: ignore if GW not in any split (shouldn't happen here)

# Convert to numpy arrays
X_train_seq = np.array(X_train_seq, dtype=np.float32)
y_train_seq = np.array(y_train_seq, dtype=np.float32)

X_val_seq = np.array(X_val_seq, dtype=np.float32)
y_val_seq = np.array(y_val_seq, dtype=np.float32)

X_test_seq = np.array(X_test_seq, dtype=np.float32)
y_test_seq = np.array(y_test_seq, dtype=np.float32)

print("\n=== Sequence dataset shapes (T = {}) ===".format(SEQ_LEN))
print("X_train_seq:", X_train_seq.shape, "| y_train_seq:", y_train_seq.shape)
print("X_val_seq  :", X_val_seq.shape,   "| y_val_seq  :", y_val_seq.shape)
print("X_test_seq :", X_test_seq.shape,  "| y_test_seq :", y_test_seq.shape)

# Quick sanity check
if X_train_seq.shape[0] > 0:
    print("\nExample train sequence shape (should be (SEQ_LEN, D)):", X_train_seq[0].shape)


Full seq_df shape: (25251, 104)
Number of features (D): 46


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(



=== Sequence dataset shapes (T = 5) ===
X_train_seq: (10698, 5, 46) | y_train_seq: (10698,)
X_val_seq  : (5074, 5, 46) | y_val_seq  : (5074,)
X_test_seq : (5489, 5, 46) | y_test_seq : (5489,)

Example train sequence shape (should be (SEQ_LEN, D)): (5, 46)


## 9. LSTM Model — Using 5-Gameweek History

So far, our **MLP** model treated each `(player, gameweek)` row as an **independent
sample**, with no explicit notion of time. However, football performance is
naturally **sequential**:

- A player’s points in gameweek *t* are influenced by what happened in the
  previous gameweeks (*t−1, t−2, …*).
- Recurrent neural networks (RNNs) like **LSTM** are designed to model such
  **time-series / sequence data**.

In Section 8, we already converted our flat table into sequence datasets:

- Each sample in:
  - `X_train_seq` has shape `(T, D)` where:
    - `T = 5` is the number of **past gameweeks** in the history window.
    - `D = 46` is the number of **features** (same as in the MLP).
  - `y_train_seq` is a scalar: the FPL points in **gameweek t** (the next GW).
- Similarly, we have `X_val_seq, y_val_seq` and `X_test_seq, y_test_seq`.

### 9.1 LSTM architecture

We now build an **LSTM-based regression model**:

- **Input**: sequence of shape `(T = 5, D = 46)` per sample.
- **LSTM layer (64 units)**: reads the 5-step history and produces a hidden
  representation summarising the player’s recent form and context.
- **Dropout(0.3)**: regularises the model (reduces overfitting).
- **Dense(32, ReLU)**: small fully-connected layer to transform the LSTM output.
- **Dense(1)**: final scalar output → predicted FPL points for the next gameweek.

### 9.2 Training setup

We train the LSTM with:

- **Loss**: Mean Squared Error (`mse`) — standard for regression.
- **Metric**: Mean Absolute Error (`mae`) — easy to interpret in “FPL points”.
- **Optimizer**: Adam with learning rate `1e-3`.

We also use two callbacks:

1. **EarlyStopping**:
   - Monitors validation MAE (`val_mae`).
   - `patience=5`: if validation MAE doesn’t improve for 5 epochs, stop training.
   - `restore_best_weights=True`: after training finishes, reload the best model.

2. **ReduceLROnPlateau**:
   - If `val_mae` stops improving for 3 epochs, reduce the learning rate by 0.5.
   - This helps the model fine-tune around a local minimum.

### 9.3 What we will do in this cell

1. Print the shapes of `X_train_seq`, `X_val_seq`, `X_test_seq`.
2. Build and summarise the LSTM model.
3. Train the model with early stopping + LR scheduling.
4. Plot **training vs validation MAE** over epochs.
5. Evaluate MAE and RMSE on:
   - Training set
   - Validation set
   - Test set

We will reuse the same `evaluate_regression(...)` helper so all models
(baseline, MLP, LSTM, GRU) are comparable on the **same metrics**.


X_train_seq shape: (10698, 5, 46)
X_val_seq shape  : (5074, 5, 46)
X_test_seq shape : (5489, 5, 46)
y_train_seq shape: (10698,)
y_val_seq shape  : (5074,)
y_test_seq shape : (5489,)
LSTM input shape: (5, 46)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        28,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,529 (119.25 KB)

 Trainable params: 30,529 (119.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 7s 35ms/step - loss: 6.0619 - mae: 1.2464 - val_loss: 3.9103 - val_mae: 1.1311 - learning_rate: 0.0010
Epoch 2/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 4.6598 - mae: 1.2041 - val_loss: 3.9089 - val_mae: 1.1071 - learning_rate: 0.0010
Epoch 3/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 4.5898 - mae: 1.1884 - val_loss: 3.9055 - val_mae: 1.1088 - learning_rate: 0.0010
Epoch 4/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 4.5643 - mae: 1.1808 - val_loss: 3.9182 - val_mae: 1.1042 - learning_rate: 0.0010
Epoch 5/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 4.4828 - mae: 1.1744 - val_loss: 3.9222 - val_mae: 1.0921 - learning_rate: 0.0010
Epoch 6/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 4.5231 - mae: 1.1700 - val_loss: 3.9430 - val_mae: 1.0870 - learning_rate: 0.0010
Epoch 7/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 4.4360 - mae: 1.1626 - val_loss: 3.9222 - val_mae: 1.0800 - learning_rate: 0.0010
Epoch 8/50
84

=== LSTM performance ===
335/335 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Train MAE : 0.9815
Train RMSE: 1.9043
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Val MAE : 1.0448
Val RMSE: 2.0370
172/172 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Test MAE : 1.1607
Test RMSE: 2.3525


(1.160685658454895, np.float64(2.352536528868239))

In [15]:
# =======================================
# 9. LSTM model: architecture & training
# =======================================

from tensorflow.keras import layers, callbacks

# 1) Sanity check: make sure sequence datasets exist and shapes look right
print("X_train_seq shape:", X_train_seq.shape)
print("X_val_seq shape  :", X_val_seq.shape)
print("X_test_seq shape :", X_test_seq.shape)
print("y_train_seq shape:", y_train_seq.shape)
print("y_val_seq shape  :", y_val_seq.shape)
print("y_test_seq shape :", y_test_seq.shape)

# Each input sample to the LSTM has shape (timesteps, num_features)
input_shape = (X_train_seq.shape[1], X_train_seq.shape[2])  # (T, D)
print("LSTM input shape:", input_shape)


def build_lstm_model(input_shape):
    """
    Build a simple LSTM-based regression model for FPL points prediction.

    input_shape: tuple (T, D)
      - T: number of time steps (history length, here 5 gameweeks)
      - D: number of features per timestep (same 46 features as MLP)
    """
    model = keras.Sequential([
        # Explicitly define the input shape: (T, D)
        layers.Input(shape=input_shape),

        # LSTM layer reads the 5-step history and outputs a single feature vector.
        # return_sequences=False → only keep the final hidden state.
        layers.LSTM(64, return_sequences=False),

        # Dropout for regularisation (reduces overfitting).
        layers.Dropout(0.3),

        # Small dense layer to further process the LSTM output.
        layers.Dense(32, activation="relu"),

        # Final regression output: a single scalar (predicted FPL points).
        layers.Dense(1)
    ])

    # Compile with MSE loss and MAE metric (for interpretability).
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss="mse",
        metrics=["mae"]
    )
    return model


# 2) Build and summarise the LSTM model
lstm_model = build_lstm_model(input_shape)
lstm_model.summary()

# 3) Define callbacks: EarlyStopping and ReduceLROnPlateau
early_stop_lstm = callbacks.EarlyStopping(
    monitor="val_mae",          # watch validation MAE
    patience=5,                 # stop if no improvement for 5 epochs
    restore_best_weights=True,  # roll back to best weights at the end
    verbose=1
)

reduce_lr_lstm = callbacks.ReduceLROnPlateau(
    monitor="val_mae",  # also based on validation MAE
    factor=0.5,         # multiply LR by 0.5 when triggered
    patience=3,         # wait 3 epochs without improvement
    min_lr=1e-5,        # do not go below this LR
    verbose=1
)

# 4) Train the LSTM model on sequence data
history_lstm = lstm_model.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_val_seq, y_val_seq),
    epochs=50,               # maximum epochs; early stopping will usually stop earlier
    batch_size=128,          # larger batch size since sequences are small (5 timesteps)
    callbacks=[early_stop_lstm, reduce_lr_lstm],
    verbose=1
)

# 5) Plot training vs validation MAE over epochs
plt.figure(figsize=(8, 4))
plt.plot(history_lstm.history["mae"], label="Train MAE")
plt.plot(history_lstm.history["val_mae"], label="Val MAE")
plt.xlabel("Epoch")
plt.ylabel("MAE")
plt.title("LSTM training vs validation MAE")
plt.legend()
plt.grid(True)
plt.show()

# 6) Evaluate the LSTM on train / val / test sets
print("\n=== LSTM performance ===")

# --- Train set ---
y_train_lstm_pred = lstm_model.predict(X_train_seq).ravel()
evaluate_regression(y_train_seq, y_train_lstm_pred, label="Train")

# --- Validation set ---
y_val_lstm_pred = lstm_model.predict(X_val_seq).ravel()
evaluate_regression(y_val_seq, y_val_lstm_pred, label="Val")

# --- Test set ---
y_test_lstm_pred = lstm_model.predict(X_test_seq).ravel()
evaluate_regression(y_test_seq, y_test_lstm_pred, label="Test")


## 10. GRU Model — Simpler Recurrent Alternative

We now try a second recurrent architecture: the **GRU (Gated Recurrent Unit)**.

Conceptually, GRUs are very similar to LSTMs:

- Both use **gates** to control how much past information is kept or forgotten.
- Both are designed to handle **sequences** and **long-term dependencies**.
- GRUs are usually **smaller and faster** because they have fewer parameters
  (no separate cell state like the LSTM).

In this notebook, the GRU will use exactly the **same sequence inputs** as the LSTM:

- `X_train_seq`, `X_val_seq`, `X_test_seq`  
  each of shape: `(num_samples, T=5, D=46)`
- Targets:
  - `y_train_seq`, `y_val_seq`, `y_test_seq` → FPL points in the **next gameweek**.

### 10.1 GRU architecture

We mirror the LSTM setup as closely as possible, just swapping LSTM → GRU:

- **Input**: `(T = 5, D = 46)`
- **GRU(64)** with `return_sequences=False`
- **Dropout(0.3)** for regularisation
- **Dense(32, ReLU)**
- **Dense(1)** output (regression: predicted FPL points)

### 10.2 Training setup

We keep the training configuration the same as for the LSTM:

- **Loss**: Mean Squared Error (`mse`)
- **Metric**: Mean Absolute Error (`mae`)
- **Optimizer**: Adam with learning rate `1e-3`
- **Callbacks**:
  - `EarlyStopping` on `val_mae` with `patience=5`, `restore_best_weights=True`
  - `ReduceLROnPlateau` on `val_mae` with `factor=0.5`, `patience=3`

This way, we can make a **fair comparison** between:

- Baseline (form-based)
- MLP (feed-forward)
- LSTM (sequence-based)
- **GRU (sequence-based)**

We will:

1. Build and summarise the GRU model.
2. Train it on `X_train_seq, y_train_seq` with validation on `X_val_seq, y_val_seq`.
3. Plot **training vs validation MAE**.
4. Evaluate MAE and RMSE on **train / val / test** using the same
   `evaluate_regression(...)` helper.


X_train_seq shape: (10698, 5, 46)
X_val_seq shape  : (5074, 5, 46)
X_test_seq shape : (5489, 5, 46)
y_train_seq shape: (10698,)
y_val_seq shape  : (5074,)
y_test_seq shape : (5489,)
GRU input shape: (5, 46)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru (GRU)                       │ (None, 64)             │        21,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,617 (92.25 KB)

 Trainable params: 23,617 (92.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 7s 30ms/step - loss: 6.1640 - mae: 1.3204 - val_loss: 4.0233 - val_mae: 1.2116 - learning_rate: 0.0010
Epoch 2/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 4.7116 - mae: 1.2234 - val_loss: 3.9603 - val_mae: 1.1728 - learning_rate: 0.0010
Epoch 3/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 4.6690 - mae: 1.2173 - val_loss: 3.9371 - val_mae: 1.1479 - learning_rate: 0.0010
Epoch 4/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 4.6421 - mae: 1.2073 - val_loss: 3.8932 - val_mae: 1.1223 - learning_rate: 0.0010
Epoch 5/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 4.6016 - mae: 1.1990 - val_loss: 3.8945 - val_mae: 1.1121 - learning_rate: 0.0010
Epoch 6/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 4.4773 - mae: 1.1915 - val_loss: 3.8978 - val_mae: 1.1093 - learning_rate: 0.0010
Epoch 7/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 4.4759 - mae: 1.1879 - val_loss: 3.8899 - val_mae: 1.0940 - learning_rate: 0.0010
Epoch 8/50
84

=== GRU performance ===
335/335 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Train MAE : 0.9930
Train RMSE: 1.9302
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Val MAE : 1.0329
Val RMSE: 1.9939
172/172 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Test MAE : 1.1709
Test RMSE: 2.3360


(1.1709117889404297, np.float64(2.3359537283547387))

In [17]:
# =======================================
# 10. GRU model: architecture & training
# =======================================

from tensorflow.keras import layers, callbacks

# Sanity check (should match what we saw for LSTM)
print("X_train_seq shape:", X_train_seq.shape)
print("X_val_seq shape  :", X_val_seq.shape)
print("X_test_seq shape :", X_test_seq.shape)
print("y_train_seq shape:", y_train_seq.shape)
print("y_val_seq shape  :", y_val_seq.shape)
print("y_test_seq shape :", y_test_seq.shape)

gru_input_shape = (X_train_seq.shape[1], X_train_seq.shape[2])
print("GRU input shape:", gru_input_shape)


def build_gru_model(input_shape):
    """
    Build a GRU-based regression model for predicting FPL points.

    input_shape: (T, D)
      - T: number of time steps (5 previous gameweeks)
      - D: number of features (46 per timestep)
    """
    model = keras.Sequential([
        # Input shape must be (timesteps, features)
        layers.Input(shape=input_shape),

        # GRU layer reads the 5-step history and outputs one vector
        layers.GRU(64, return_sequences=False),

        # Dropout for regularisation
        layers.Dropout(0.3),

        # Small dense layer after the recurrent layer
        layers.Dense(32, activation="relu"),

        # Output: predicted FPL points (scalar)
        layers.Dense(1)
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss="mse",
        metrics=["mae"]
    )
    return model


# 1) Build and summarise the GRU model
gru_model = build_gru_model(gru_input_shape)
gru_model.summary()

# 2) Callbacks: same logic as for LSTM but separate instances
early_stop_gru = callbacks.EarlyStopping(
    monitor="val_mae",
    patience=5,
    restore_best_weights=True,
    verbose=1
)

reduce_lr_gru = callbacks.ReduceLROnPlateau(
    monitor="val_mae",
    factor=0.5,
    patience=3,
    min_lr=1e-5,
    verbose=1
)

# 3) Train the GRU model
history_gru = gru_model.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_val_seq, y_val_seq),
    epochs=50,
    batch_size=128,
    callbacks=[early_stop_gru, reduce_lr_gru],
    verbose=1
)

# 4) Plot training vs validation MAE
plt.figure(figsize=(8, 4))
plt.plot(history_gru.history["mae"], label="Train MAE")
plt.plot(history_gru.history["val_mae"], label="Val MAE")
plt.xlabel("Epoch")
plt.ylabel("MAE")
plt.title("GRU training vs validation MAE")
plt.legend()
plt.grid(True)
plt.show()

# 5) Evaluate GRU on train / val / test sets
print("\n=== GRU performance ===")

# --- Train ---
y_train_gru_pred = gru_model.predict(X_train_seq).ravel()
evaluate_regression(y_train_seq, y_train_gru_pred, label="Train")

# --- Validation ---
y_val_gru_pred = gru_model.predict(X_val_seq).ravel()
evaluate_regression(y_val_seq, y_val_gru_pred, label="Val")

# --- Test ---
y_test_gru_pred = gru_model.predict(X_test_seq).ravel()
evaluate_regression(y_test_seq, y_test_gru_pred, label="Test")


## 11. Comparing All Models: Baseline vs MLP vs LSTM vs GRU

So far, we have trained **four different predictors** of next–gameweek FPL points:

1. **Baseline (Last-3-GWs Average)**  
   - For each (player, gameweek) we simply predict  
     `event_points(t) ≈ avg_points_last3(t)` –  
     i.e. the average of that player’s previous 3 gameweeks.  
   - This is a realistic **form-based heuristic** and acts as a strong baseline
     that all other models should try to beat.

2. **MLP (Feed-forward neural network)**  
   - Input: one **flat feature vector** per (player, gameweek) of length 46:
     - lag features: `event_points_prev1`, `avg_points_last3`, rolling xGI, rolling ICT  
     - popularity & value: transfers in/out, selected%, `value_form`, `value_season`  
     - team strength & Elo  
     - one-hot encodings of **position** and **team**
   - Output: a single scalar with the predicted FPL points in the next GW.  
   - The MLP learns **non-linear combinations** of these engineered features, but
     it does **not directly model sequences**.

3. **LSTM (Sequence model, T = 5)**  
   - Input: for each target GW `t`, we feed a **sequence** of the last `T = 5`
     time steps:
     \[
       [\text{features}(t-5),\ \text{features}(t-4),\dots,\text{features}(t-1)]
     \]
     → shape: `(5, 46)` per sample.  
   - The LSTM processes the sequence step by step and can learn patterns such as:
     - gradual improvement / decline in form,
     - rotation risk (many low-minute games),
     - persistent high involvement (xGI, ICT, etc.).
   - Output: one scalar prediction for `event_points(t)`.

4. **GRU (Sequence model, T = 5)**  
   - Similar to the LSTM but using a **Gated Recurrent Unit (GRU)** cell.  
   - GRUs are usually lighter (fewer parameters) and can sometimes converge
     faster or generalise slightly differently.

---

### 11.1 Why we compare them systematically

Each model was trained on the same **time-based splits**:

- **Train GWs**: model sees these during training.  
- **Validation GWs**: used for early stopping and hyperparameter choices.  
- **Test GWs**: completely held out – used only once at the end to estimate
  real-world performance.

To compare models fairly we:

1. Evaluate **all models** on **Train**, **Validation**, and **Test** splits.
2. Use consistent error metrics:
   - **MAE (Mean Absolute Error)** – average absolute difference in FPL points.
   - **RMSE (Root Mean Squared Error)** – like MAE but penalises larger errors
     more strongly.
3. Summarise everything in a single **comparison table**:
   - Columns: `Model`, `Split`, `MAE`, `RMSE`
   - Rows: Baseline, MLP, LSTM, GRU × (Train / Val / Test)

This lets us answer:

- Does the MLP actually beat the simple form-based baseline?
- Do sequence models (LSTM / GRU) bring extra value over static features?
- Is any model clearly **overfitting** (very low Train error but much worse Test
  error)?

---

### 11.2 What this cell does

This code cell will:

1. Define a helper function `get_metrics(y_true, y_pred)` to compute MAE and RMSE
   (we compute RMSE as the square root of MSE to stay compatible with this
   `sklearn` version).
2. For each model (Baseline, MLP, LSTM, GRU):
   - Generate predictions on **Train**, **Val**, and **Test** splits.
   - Compute MAE and RMSE.
   - Append a row with `(Model, Split, MAE, RMSE)` into a list.
3. Convert all rows into a **Pandas DataFrame** `summary_df` and display it.
4. Plot a simple **bar chart of Test MAE** to visually compare how each model
   performs on unseen gameweeks (lower bar = better).

We will later use this table and plot in the **Results & Discussion** section of
the report to interpret which modelling approach works best for predicting FPL
player points.


=== Model comparison (lower is better) ===


,Model,Split,MAE,RMSE
0,MLP,Test,1.052190,2.566617
1,Baseline (last3 avg),Test,1.133573,2.357124
2,LSTM,Test,1.160686,2.352537
3,GRU,Test,1.170912,2.335954
4,MLP,Train,0.830069,1.670080
5,LSTM,Train,0.981475,1.904286
6,GRU,Train,0.992977,1.930169
7,Baseline (last3 avg),Train,1.126227,2.244513
8,MLP,Val,0.954807,2.293378
9,GRU,Val,1.032943,1.993903


In [19]:
# =========================================
# 11. Compare Baseline, MLP, LSTM, and GRU
# =========================================

from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

def get_metrics(y_true, y_pred):
    """
    Compute MAE and RMSE from true and predicted values.

    NOTE: We compute RMSE manually as sqrt(MSE) to avoid using the
    'squared=' argument, which is not supported in this sklearn version.
    """
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)  # default: squared=True
    rmse = np.sqrt(mse)
    return mae, rmse


summary_rows = []

# -------------------------------------------------
# 1) Baseline model: last-3-gameweeks average
# -------------------------------------------------
# For each split we simply use avg_points_last3 as the prediction.
# y_*_true were already defined earlier as the event_points in each split.

baseline_train_pred = train_df["avg_points_last3"].values
baseline_val_pred   = val_df["avg_points_last3"].values
baseline_test_pred  = test_df["avg_points_last3"].values

for split_name, y_true, y_pred in [
    ("Train", y_train_true, baseline_train_pred),
    ("Val",   y_val_true,   baseline_val_pred),
    ("Test",  y_test_true,  baseline_test_pred),
]:
    mae, rmse = get_metrics(y_true, y_pred)
    summary_rows.append({
        "Model": "Baseline (last3 avg)",
        "Split": split_name,
        "MAE": mae,
        "RMSE": rmse,
    })

# --------------------------
# 2) MLP: flat feature model
# --------------------------
# Use the trained MLP to predict on the scaled flat feature matrices.

mlp_train_pred = mlp_model.predict(X_train_scaled, verbose=0).ravel()
mlp_val_pred   = mlp_model.predict(X_val_scaled,   verbose=0).ravel()
mlp_test_pred  = mlp_model.predict(X_test_scaled,  verbose=0).ravel()

for split_name, y_true, y_pred in [
    ("Train", y_train, mlp_train_pred),
    ("Val",   y_val,   mlp_val_pred),
    ("Test",  y_test,  mlp_test_pred),
]:
    mae, rmse = get_metrics(y_true, y_pred)
    summary_rows.append({
        "Model": "MLP",
        "Split": split_name,
        "MAE": mae,
        "RMSE": rmse,
    })

# -------------------------------
# 3) LSTM: sequence (T = 5) model
# -------------------------------
# Use the sequence inputs (5 timesteps x 46 features) for LSTM evaluation.

lstm_train_pred = lstm_model.predict(X_train_seq, verbose=0).ravel()
lstm_val_pred   = lstm_model.predict(X_val_seq,   verbose=0).ravel()
lstm_test_pred  = lstm_model.predict(X_test_seq,  verbose=0).ravel()

for split_name, y_true, y_pred in [
    ("Train", y_train_seq, lstm_train_pred),
    ("Val",   y_val_seq,   lstm_val_pred),
    ("Test",  y_test_seq,  lstm_test_pred),
]:
    mae, rmse = get_metrics(y_true, y_pred)
    summary_rows.append({
        "Model": "LSTM",
        "Split": split_name,
        "MAE": mae,
        "RMSE": rmse,
    })

# ------------------------------
# 4) GRU: sequence (T = 5) model
# ------------------------------
# Same sequence inputs as for the LSTM, but using the GRU model.

gru_train_pred = gru_model.predict(X_train_seq, verbose=0).ravel()
gru_val_pred   = gru_model.predict(X_val_seq,   verbose=0).ravel()
gru_test_pred  = gru_model.predict(X_test_seq,  verbose=0).ravel()

for split_name, y_true, y_pred in [
    ("Train", y_train_seq, gru_train_pred),
    ("Val",   y_val_seq,   gru_val_pred),
    ("Test",  y_test_seq,  gru_test_pred),
]:
    mae, rmse = get_metrics(y_true, y_pred)
    summary_rows.append({
        "Model": "GRU",
        "Split": split_name,
        "MAE": mae,
        "RMSE": rmse,
    })

# ---------------------------------
# Build and display comparison table
# ---------------------------------
summary_df = pd.DataFrame(summary_rows)
summary_df = summary_df.sort_values(["Split", "MAE"]).reset_index(drop=True)

print("=== Model comparison (lower is better) ===")
display(summary_df)

# -------------------------------
# Bar chart of Test MAE by model
# -------------------------------
test_df_metrics = summary_df[summary_df["Split"] == "Test"]

plt.figure(figsize=(6, 4))
plt.bar(test_df_metrics["Model"], test_df_metrics["MAE"])
plt.ylabel("Test MAE (FPL points)")
plt.title("Test MAE by model")
plt.xticks(rotation=20)
plt.grid(axis="y", alpha=0.3)
plt.tight_layout()
plt.show()


## 12. Interpreting the Results: Baseline vs MLP vs LSTM vs GRU

### 12.1 Recap of the experiment

- **Target**: Predict each player's FPL points in gameweek *t* using information up to gameweek *t−1*.
- **Data**: 2024–25 season, one row per *(player, gameweek)*.
- **Features**:
  - Lag & form: `event_points_prev1`, `avg_points_last3`, rolling xGI per 90, rolling ICT.
  - Popularity & value: `transfers_in_event`, `transfers_out_event`, `selected_by_percent`,
    `value_form`, `value_season`.
  - Team context: strength ratings + Elo.
  - Categorical: position and team (one-hot encoded).
- **Splits (time-based)**:
  - Train: GWs 4–24
  - Val:   GWs 25–31
  - Test:  GWs 32–38

We trained four models:

1. **Baseline** – predict `avg_points_last3` (last-3-GWs moving average).
2. **MLP** – feed-forward network on the flat feature vector (46 features).
3. **LSTM** – sequence model on last 5 gameweeks (shape: 5×46).
4. **GRU** – sequence model on last 5 gameweeks (shape: 5×46).

We evaluate with:

- **MAE** (Mean Absolute Error) – average absolute mistake in FPL points.
- **RMSE** (Root Mean Squared Error) – similar but penalises big errors more.

---

### 12.2 Test-set comparison (unseen gameweeks)

On the **Test split** (GWs 32–38), we obtained:

| Model                  | Test MAE | Test RMSE |
|------------------------|---------:|----------:|
| **MLP**                | **1.05** | 2.57      |
| Baseline (last3 avg)   | 1.13     | 2.36      |
| LSTM                   | 1.16     | 2.35      |
| GRU                    | 1.17     | **2.34**  |

**Key observations:**

1. **MLP has the lowest Test MAE (≈ 1.05)**  
   → On average, the MLP’s prediction is off by about **1 FPL point**.  
   → It improves on the baseline’s MAE (~1.13) by ~0.08 points (~7% better).

2. **GRU has the lowest Test RMSE (≈ 2.34)**  
   → GRU and LSTM have slightly **lower RMSE** than the baseline and clearly lower than the MLP.  
   → This means that, although their average absolute error is higher, they
     seem to handle **big outliers** (very good or very bad scores) a bit more
     gracefully than the MLP.

3. **MLP’s higher RMSE**  
   - MAE is best, but **RMSE is worst** among the four.
   - Interpretation:
     - For “typical” cases, the MLP predicts very well (low MAE).
     - Occasionally it makes **larger mistakes** (e.g. badly missing a haul or a blank),
       which inflate the RMSE.

So you can say:

> “Our best **average** predictor is the MLP, but if we care a lot about
> avoiding **big mistakes**, the GRU / LSTM are slightly more conservative.”

---

### 12.3 Train vs Validation vs Test (overfitting / generalisation)

Looking at MAE across splits:

- **Baseline (last3 avg)**  
  - Train: 1.13  
  - Val:   1.13  
  - Test:  1.13  
  → Very stable, as expected from a simple moving average.

- **MLP**  
  - Train: 0.83  
  - Val:   0.95  
  - Test:  1.05  
  → There is a **gap** between Train and Test (0.83 → 1.05):  
    - MLP is more flexible and slightly **overfits** the training data.  
    - However, it still generalises well enough to beat the baseline on MAE.

- **LSTM**  
  - Train: 0.98  
  - Val:   1.04  
  - Test:  1.16  
  → Reasonable train/val match, but a bigger drop on Test.  
    - LSTM is capturing some sequential signal (better than the baseline on Val),
      but struggles more on the last GWs.

- **GRU**  
  - Train: 0.99  
  - Val:   1.03  
  - Test:  1.17  
  → Similar story to LSTM: decent on Train/Val, slightly worse on Test.

**Takeaway for the presentation:**

- **Static engineered features + MLP already capture most of the signal** in the
  current dataset (short season, limited history per player).
- Sequence models (LSTM / GRU) do not yet bring a clear improvement in MAE, but
  they slightly **stabilise extreme errors** (better RMSE).
- Differences are **not huge** (all Test MAEs are in the 1.05–1.17 range) – so
  model choice might depend on what metric we care about.

---

### 12.4 How to narrate this in words

You can summarise it like:

> “We compared a realistic form-based baseline with three neural models:
>  an MLP that uses engineered features, and two sequence models (LSTM/GRU)
>  that consume the last 5 gameweeks of data per player.
>
>  On held-out gameweeks, the MLP achieved the lowest Mean Absolute Error
>  (~1.05 FPL points), outperforming both the baseline (~1.13) and the
>  recurrent models (~1.16–1.17). This suggests that our engineered features —
>  especially lagged points, rolling form, and team strength — already encode
>  most of the predictive signal for typical matches.
>
>  However, the GRU and LSTM slightly improve the RMSE compared to the baseline
>  and MLP, indicating that they may handle extreme outcomes (big hauls or
>  unexpected blanks) a little better.
>
>  Overall, in this setup, a relatively simple MLP with good feature engineering
>  is sufficient to match or outperform more complex sequence models on average
>  error, although sequence models offer some robustness to outliers.”

---

### 12.5 Limitations (good to mention in the report)

- **Short sequences (T = 5)**:  
  We only give the last 5 gameweeks. A player’s role or form can change over
  longer horizons (injury, transfer, tactical change).
- **Target noisiness**:  
  FPL points are noisy (penalties, cards, lucky goals). Even a perfect model
  of “performance” can’t capture all randomness.
- **Fixture context**:  
  Our features don’t yet include *who* the opponent is in the upcoming GW
  (e.g. facing MCI away vs SHU at home).
- **Minutes / start probability**:  
  We don’t explicitly model whether the player will start or their expected
  minutes; this strongly affects points.

---

### 12.6 Next steps in the notebook & project

Here’s what I’d recommend doing next:

1. **Error analysis & visualisations**
   - Plot **true vs predicted** points for each model on the Test set.
   - Plot **histograms of residuals** (error = y_true − y_pred) to see if some
     models are more biased.
   - Maybe break down errors by:
     - Position (GK / DEF / MID / FWD).
     - Player price brackets.
     - Team strength tiers.

2. **Per-player case studies**
   - Pick a few well-known players (e.g. Haaland, Salah, Saka).
   - Plot their **actual vs predicted** points over time for each model.
   - This gives a very concrete story for your presentation.

3. **Feature importance / sensitivity (for MLP)**
   - Even though it’s a NN, you can:
     - Inspect correlations between each feature and target.
     - Try simple *feature ablation*: retrain MLP without certain feature groups
       (e.g. without team strength) and see how MAE changes.

4. **Model improvements (if you have time)**
   - Add richer features:
     - Upcoming opponent strength, home/away, implied odds (if available).
     - Binary flag for “played 60+ minutes” in previous GWs.
   - Try different sequence lengths (`SEQ_LEN = 3` vs 7 vs 10) for LSTM/GRU and
     see if that helps.
   - Slight tuning of network sizes / dropout for MLP and LSTM/GRU.

5. **Wrap up for the report**
   - Add a **Conclusion** markdown section summarising:
     - Problem statement.
     - Best model and metrics.
     - Key qualitative insights.
     - Limitations & future work.
   - Optionally, save `summary_df` as a CSV and include a nice formatted table
     in your write-up.

---

If you’d like, next we can:

- add an **“Error analysis” section cell** (markdown + code) to draw true vs predicted
  scatter plots and residual histograms for the Test set, or  
- write the **Conclusion & Future Work** markdown that you can paste directly into the notebook / report.
::contentReference[oaicite:0]{index=0}
